# ロイターニュースのトップニュースをスクライピング

スクライピングのテスト

1. 必要なライブラリをインポート。最低でもRequestとBeautifulSoup
2. Requesstを使って指定のサイトからHtmlをとってくる
3. 取ってきたHtmlをBS4を使ってタグで呼び出せるようにする
4. 欲しい情報のあるタグを探す
5. 見つけた情報の表示とCVSファイルに保存

In [2]:
# Requests: WebページのHTMLを取得
import requests
# Beautiful Soupのインポート
from bs4 import BeautifulSoup
# Pandas をインポート
import pandas as pd 

In [10]:
# requestsを使ってURLからデータを取得
reuters = requests.get("https://jp.reuters.com/news/topNews").text
# BeautifulSoupの初期化
soup = BeautifulSoup(reuters, 'html.parser')
# このページのh2タグをすべて抜き出す
tags = soup.find_all("h2")

In [21]:
def check_news():
    # タイトルコラムの作成
    title_columns = ["Name", "Post Date", "URL"]
    # 列（コラム）を作成
    columns = pd.DataFrame(columns=title_columns)
    # 結果が何も表示されない時のため
    result = False  
    # カウントするための変数の作成
    count = 0

    # インプットを作成
    key_word = input("知りたいニュースのキーワードを入力してください : ")
    
    # ここからtags = soup.find_all("h2")をループさせて出力
    for tag in tags:
        # a tagを探す
        ahref = tag.find("a")
        # timestampのあるデータを見つける
        time = tag.find("span", {"class": "timestamp"})
        # 記事名を取得
        name = ahref.string  
        url = "https://jp.reuters.com" + ahref.get("href")  # URLを取得

        # キーワードを含むものを探す
        if str(key_word) in str(ahref.string):
            # 時間の情報あるものとないもので分ける。これしないとエラーが出たので作りました。
            if not time is None:
                print("---------------------------------------")
                print(name)
                print("投稿日時",time.string)
                print(url)
                print("---------------------------------------")
                rows = pd.Series([name, time.string, url], title_columns)  # 行を作成
                columns = columns.append(rows, title_columns)  # データフレームに行を追加
                count += 1  # 入手できた投稿数をカウント
            elif not ahref.string is None:
                print("---------------------------------------")
                print(name)
                print(url)
                print("---------------------------------------")
                rows = pd.Series([name, None, url], title_columns)  # 行を作成
                columns = columns.append(rows, title_columns)  # データフレームに行を追加
                count += 1 # 入手できた投稿数をカウント
            result = True

    # 結果が何も表示されない時はこれを表示
    if result is False:
        print("トップニュースには『" + key_word + "』に関する記事はありませんでした。")
        
        
    return columns, count

In [24]:
columns, count = check_news()

知りたいニュースのキーワードを入力してください : 中国
---------------------------------------
コラム：中国怒らせた欧米高級ブランド、謝罪して不買のジレンマ
投稿日時 8:56am JST
https://jp.reuters.com/article/breakingviews-column-luxury-china-idJPKCN1V50II
---------------------------------------
---------------------------------------
台湾、来年防衛費8.3％の大幅増額　中国の脅威に対抗
投稿日時 8:34am JST
https://jp.reuters.com/article/taiwan-china-defence-idJPKCN1V52LN
---------------------------------------
---------------------------------------
トランプ氏　「中国はまず香港情勢に対処を」　貿易問題と絡める（字幕・15日）
https://jp.reuters.com/video/2019/08/15/%E3%83%88%E3%83%A9%E3%83%B3%E3%83%97%E6%B0%8F%E3%80%80%E3%80%8C%E4%B8%AD%E5%9B%BD%E3%81%AF%E3%81%BE%E3%81%9A%E9%A6%99%E6%B8%AF%E6%83%85%E5%8B%A2%E3%81%AB%E5%AF%BE%E5%87%A6%E3%82%92%E3%80%8D%E3%80%80%E8%B2%BF%E6%98%93%E5%95%8F%E9%A1%8C%E3%81%A8%E7%B5%A1%E3%82%81%E3%82%8B%E5%AD%97%E5%B9%95%E3%83%BB15%E6%97%A5?videoId=587701793&videoChannel=201
---------------------------------------
---------------------------------------
コラム：中国怒らせた欧米高級ブランド、謝罪して不買のジレンマ
ht

In [25]:
#ここからは出た結果をCSVファイルとして保存するコード
#fileの作成
filename = "result.csv"
#作成したファイルにcheck_newsで作ったcolumnsのファイルを入れる
columns.to_csv(filename, encoding='utf-8-sig')
# テストでカウント数分だけ出力
columns.head(count) 

,Name,Post Date,URL
0,コラム：中国怒らせた欧米高級ブランド、謝罪して不買のジレンマ,8:56am JST,https://jp.reuters.com/article/breakingviews-c...
1,台湾、来年防衛費8.3％の大幅増額 中国の脅威に対抗,8:34am JST,https://jp.reuters.com/article/taiwan-china-de...
2,トランプ氏 「中国はまず香港情勢に対処を」 貿易問題と絡める（字幕・15日）,None,https://jp.reuters.com/video/2019/08/15/%E3%83...
3,コラム：中国怒らせた欧米高級ブランド、謝罪して不買のジレンマ,None,https://jp.reuters.com/article/breakingviews-c...
